# Kaggle house price prediction problem

Description of the data 
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

1. Load data_combined_cleaned.csv 
2. Filter out all records where SalePrice is nan
3. Perform one hot encoding on full data set
4. Remove Id column from dataset
5. Divide the data into training and test datasets, use test size = 0.3 and random state = 1
6. Create pipeline to scale the data and fit model
7. Find r2 score based on training data and testing data

Cleaned dataset is located in link below
https://github.com/abulbasar/data/tree/master/kaggle-houseprice


In [3]:
import pandas as pd
import numpy as np
from sklearn import *
import matplotlib.pyplot as plt

In [17]:
df = pd.read_csv("/data/kaggle/house-prices/data_combined_cleaned.csv")
df = df[~df.SalesPrice.isnull()]
del df["Id"]

y = df.SalesPrice
X = df.copy()
del X["SalesPrice"]
X_dummy = pd.get_dummies(X, drop_first=True)
X_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Columns: 258 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(11), int64(25), uint8(222)
memory usage: 738.6 KB


In [20]:
%%time 

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy
                        , y, test_size = 0.3, random_state = 1230)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 1, 
                                include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha = 450, tol=0.0001) )
])

pipe.fit(X_train, y_train)
print("train R2", pipe.score(X_train, y_train), 
      "test R2:", pipe.score(X_test, y_test))


train R2 0.918867817322 test R2: 0.838123384039
CPU times: user 74.3 ms, sys: 5.24 ms, total: 79.5 ms
Wall time: 78.4 ms


In [23]:
scores = model_selection.cross_val_score(pipe, 
                    X_dummy, y, cv = 5, verbose=True)

np.mean(scores)

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s finished


0.82456069007898147

In [29]:
pipe1 = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 1, 
                                include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha = 450, tol=0.0001) )
])

pipe2 = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree = 1, 
                                include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha = 40, tol=0.0001) )
])

scores1 = model_selection.cross_val_score(pipe1, 
                    X_dummy, y, cv = 5, verbose=True)

scores2 = model_selection.cross_val_score(pipe2, 
                    X_dummy, y, cv = 5, verbose=True)


np.mean(scores1), np.mean(scores2)

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


(0.82456069007898147, 0.8179883345147877)

In [42]:
%%time
param_grid = {
    "est__alpha": [500, 700, 850, 800, 970], 
    "est__tol": 10 ** np.linspace(-2, -3, 10)
}
gs = model_selection.GridSearchCV(param_grid= param_grid, 
                        estimator=pipe1, cv = 5, verbose=True)
gs.fit(X_dummy, y)
print("Best param", gs.best_params_, "best score: ", gs.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best param {'est__alpha': 970, 'est__tol': 0.0016681005372000592} best score:  0.832396031775
CPU times: user 8.9 s, sys: 391 ms, total: 9.29 s
Wall time: 9.29 s


[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:    9.2s finished


In [43]:
gs.best_estimator_

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', Lasso(alpha=970, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0016681005372000592, warm_start=False))])

In [34]:
gs.best_params_

{'est__alpha': 1000}

In [35]:
gs.best_score_

0.83233698641556486

In [44]:
import pickle

In [47]:
with open("house_price_v1.obj", "wb") as f:
    pickle.dump(gs.best_estimator_, f)

In [49]:
!ls -l

total 1368
-rw-r--r--  1 abulbasar  staff  169612 Jan 30 11:35 D1 - 01 pandas dataframe.ipynb
-rw-r--r--  1 abulbasar  staff   33728 Jan 30 12:06 D1 - 02 top 10 movies (dataframe ops).ipynb
-rw-r--r--  1 abulbasar  staff  189551 Jan 30 13:04 D1 - 03 visualization.ipynb
-rw-r--r--  1 abulbasar  staff   51525 Jan 30 15:44 D1 - 04 Regression (Insurance).ipynb
-rw-r--r--  1 abulbasar  staff   47399 Jan 30 17:26 D1 - 05 Exercise 2.ipynb
-rw-r--r--  1 abulbasar  staff  170324 Jan 31 12:56 Day 2 - 01 Exercise - Kaggle house price prediction .ipynb
-rw-r--r--  1 abulbasar  staff    8144 Jan 31 14:52 Day 2 - 02 Cross validation and hyper parameter tuning.ipynb
-rw-r--r--  1 abulbasar  staff       8 Jan 30 11:21 README.md
-rw-r--r--  1 abulbasar  staff    9484 Jan 31 14:54 house_price_v1.obj


In [50]:
with open("house_price_v1.obj", "rb") as f:
    tuned_mode = pickle.load(f)

In [51]:
tuned_mode

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', Lasso(alpha=970, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0016681005372000592, warm_start=False))])

In [52]:
tuned_mode.score(X_dummy, y)

0.88995349937618362